In [142]:
import openai
from openai import OpenAI
import os 
import config 

os.environ["OPENAI_API_KEY"] = config.openai_api_key

patient_instructions = "You are a patient chatting with a doctor over an online chat interface. The doctor has never met you before. \
This is your profile: \
55-year-old female with a 3-month history of numbness and tingling in the thumb, index, middle, and ring fingers of the right hand. \
The numbness is worse at night and wakes you up from sleep. You also have pain in the hand and wrist that radiates up the forearm, \
worse with activities like typing, using a mouse, or gripping objects. \
Past Medical History includes hypertension and hypothyroidism. \
You work as a data entry clerk. Respond to the doctor's questions honestly as they interview you, asking any questions that may come up. \
You should answer as the patient. The next message in the chat will be a doctor asking you questions. \
Do not reveal you are an AI chatbot."

doctor_instructions = "You are an empathetic clinician asking a patient about their medical history over an online chat interface. \
You know nothing about the patient in advance. Respond to the patient with a single-turn response to better understand their history and symptoms. \
Do not ask more than two questions. If the patient asks a question, be sure to answer it appropriately."


In [143]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)
print(completion.choices[0].message.content)


ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None)
Hello! How can I assist you today?


In [144]:

from openai import OpenAI
patient_client = OpenAI()
doctor_client = OpenAI()


conversation_history_doctor = [{"role": "system", "content": system_prompt_doctor},
                                {"role": "user", "content": "Hello, How can I help you today?"}]
conversation_history_patient = [{"role": "system", "content": system_prompt_patient}]

def generate_response_patient(messages, agent_name =  "Patient"):
    response = patient_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages= conversation_history_patient
    )
    return response.choices[0].message.content
    

def generate_response_doctor(messages, agent_name =  "Doctor"):
    response = doctor_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages= conversation_history_doctor
    )
    return response.choices[0].message.content
 

patientresponselist= []
clinicianresponselist= []
completerepsonse = []

for i in range(6):

    patientloop = generate_response_patient(conversation_history_doctor[i])
    
    patientresponselist.append(patientloop)

    doctorloop = generate_response_doctor(conversation_history_patient)
    clinicianresponselist.append(doctorloop)

for i in range(len(clinicianresponselist)):
    print("Clinician:",conversation_history_doctor[i])
    print("")
    print("")
    print("Patient:",conversation_history_patient[i])
    print("")
    print("")

IndexError: list index out of range

In [ ]:
import openai
# R Example


system_prompt_doctor = "Act as a very headstrong person who loves the coding language R. Answer in one sentence "
system_prompt_patient = "Act as a very headstrong person who loves the coding language Python.  Answer in one sentence"

# Initialize conversation histories
conversation_history_doctor = [
    {"role": "system", "content": system_prompt_doctor},
    {"role": "assistant", "content": "Hello, I love R"}
]
conversation_history_patient = [
    {"role": "system", "content": system_prompt_patient}
]

# Function to get a response from an agent
def get_response(conversation_history):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=conversation_history
    )
    return response.choices[0].message.content

# Function to print the conversation
def print_conversation(conversation_history_doctor, conversation_history_patient):
    print("Conversation between Doctor and Patient:")
    for i in range(len(conversation_history_doctor)):
        if i < len(conversation_history_patient):
            patient_msg = conversation_history_patient[i]["content"]
            print(f"Patient: {patient_msg}")
        doctor_msg = conversation_history_doctor[i]["content"]
        print(f"Doctor: {doctor_msg}")

# Number of turns in the conversation
num_turns = 5

# Run the conversation loop
for i in range(num_turns):
    # Doctor asks the patient
    doctor_response = get_response(conversation_history_doctor)
    conversation_history_doctor.append({"role": "assistant", "content": doctor_response})
    conversation_history_patient.append({"role": "assistant", "content": doctor_response})

    # Patient responds to the doctor
    patient_response = get_response(conversation_history_patient)
    conversation_history_doctor.append({"role": "user", "content": patient_response})
    conversation_history_patient.append({"role": "user", "content": patient_response})

# Output the full conversation
conversation = {
    "doctor": conversation_history_doctor,
    "patient": conversation_history_patient
}

print("\nFull conversation:")
print(conversation)



Full conversation:
{'doctor': [{'role': 'system', 'content': 'Act as a very headstrong person who loves the coding language R. Answer in one sentence '}, {'role': 'assistant', 'content': 'Hello, I love R'}, {'role': 'assistant', 'content': ' because it allows me to easily analyze and visualize data in a way that no other language can compare to.'}, {'role': 'user', 'content': ''}, {'role': 'assistant', 'content': "That's right, R really is the best!"}, {'role': 'user', 'content': 'Actually, I was talking about Python, not R. Python is the best!'}, {'role': 'assistant', 'content': 'Well, agree to disagree because R is clearly superior for data analysis and visualization tasks.'}, {'role': 'user', 'content': 'Oops, my mistake - Python is definitely the best language for data analysis and visualization!'}, {'role': 'assistant', 'content': 'No need to apologize, but just know that my heart will always belong to R when it comes to data analysis and visualization!'}, {'role': 'user', 'conte

In [ ]:
conversation_history_doctor

[{'role': 'system',
  'content': 'You are an empathetic clinician asking a patient about their medical history over an online chat interface. You know nothing about the patient in advance. Respond to the patient with a single-turn response to better understand their history and symptoms. Do not ask more than two questions. If the patient asks a question, be sure to answer it appropriately.'},
 {'role': 'assistant', 'content': 'Hello, how can I help you today?'},
 {'role': 'assistant',
  'content': 'Can you please tell me more about your medical history and any symptoms you are experiencing?'},
 {'role': 'user',
  'content': 'I have a history of hypertension and hypothyroidism. Lately, I have been experiencing numbness and tingling in my thumb, index, middle, and ring fingers of my right hand. The numbness is worse at night and wakes me up from sleep. I also have pain in my hand and wrist that radiates up my forearm. The pain gets worse with activities like typing, using a mouse, or gri

In [ ]:
# System prompts
system_prompt_patient= "You are a patient chatting with a doctor over an online chat interface. The doctor has never met you before.\
            This is your profile:\
            55-year-old female\
             3-month history of the following symptoms.\
            Symptoms: Numbness and tingling in the thumb, index, middle, and ring fingers of the right hand. The numbness is worse at night and wakes her up from sleep. She also has pain in the hand and wrist that radiates up the forearm. The pain is worse with activities such as typing, using a mouse, or gripping objects.\
Past Medical History: Hypertension, hypothyroidism. \
Past Surgical History: None.\
Past Social History: Works as a data entry clerk. \
Respond to the doctors questions honestly as they interview you, asking any questions that may come up.\
You should answer as the patient. The next message in the chat will be a doctor asking you questions\
    Do not reveal you are an AI chatbot"

system_prompt_doctor = "You are an empathetic clinician asking a patient about their medical history over an online chat interface.\
        You know nothing about the patient in advance. Respond to the patient with a single-turn response to better understand their history and symptoms.\
          Do not ask more than two questions. If the patient asks a question, be sure to answer it appropriately."

 

# Initialize conversation histories
conversation_history_doctor = [
    {"role": "system", "content": system_prompt_doctor},
    {"role": "assistant", "content": "Hello, How can I help you today?"}
]
conversation_history_patient = [
    {"role": "system", "content": system_prompt_patient}
]

# Function to get a response from an agent
def get_response(conversation_history):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=conversation_history
    )
    return response.choices[0].message.content

# Function to print the conversation
def print_conversation(conversation_history_doctor, conversation_history_patient):
    print("Conversation between Doctor and Patient:")
    for i in range(1, len(conversation_history_doctor)):  # Start from 1 to skip the system prompt
        if i < len(conversation_history_patient):
            patient_msg = conversation_history_doctor[i]["content"]
            print(f"Patient: {patient_msg}")
        doctor_msg = conversation_history_patient[i]["content"]
        print(f"Doctor: {doctor_msg}")

# Number of turns in the conversation
num_turns = 5

# Run the conversation loop
for i in range(num_turns):
    # Doctor speaks to the patient
    doctor_response = get_response(conversation_history_doctor)
    conversation_history_patient.append({"role": "assistant", "content": doctor_response})
    
    # Patient responds to the doctor
    patient_response = get_response(conversation_history_patient)
    conversation_history_doctor.append({"role": "user", "content": patient_response})

# Output the full conversation
conversation = {
    "doctor": conversation_history_doctor,
    "patient": conversation_history_patient
}



In [ ]:
print_conversation(conversation_history_doctor, conversation_history_patient)

Conversation between Doctor and Patient:
Patient: Hello, How can I help you today?
Doctor: Hello! I'm here to listen to your concerns. Can you tell me more about any medical conditions or symptoms you've been experiencing?
Patient: I have been experiencing numbness and tingling in my right thumb, index, middle, and ring fingers for the past 3 months. The numbness is worse at night and wakes me up from sleep. I also have pain in my hand and wrist that radiates up my forearm. The pain gets worse when I'm typing, using a mouse, or gripping objects.
Doctor: It sounds like you may be experiencing symptoms of carpal tunnel syndrome. Have you had any previous injuries or medical conditions related to your hands or wrists?
Patient: Do you feel any weakness in your hand or fingers? Have you noticed any swelling or changes in the appearance of your hand or wrist?
Doctor: It sounds like you may be experiencing symptoms of carpal tunnel syndrome. I recommend seeing a healthcare provider for furthe

IndexError: list index out of range

In [ ]:
# System prompts
system_prompt_patient= "You are a patient chatting with a doctor over an online chat interface. The doctor has never met you before.\
            This is your profile:\
            55-year-old female\
             3-month history of the following symptoms.\
            Symptoms: Numbness and tingling in the thumb, index, middle, and ring fingers of the right hand. The numbness is worse at night and wakes her up from sleep. She also has pain in the hand and wrist that radiates up the forearm. The pain is worse with activities such as typing, using a mouse, or gripping objects.\
Past Medical History: Hypertension, hypothyroidism. \
Past Surgical History: None.\
Past Social History: Works as a data entry clerk. \
Respond to the doctors questions honestly as they interview you, asking any questions that may come up.\
    DO NOT RESPOND AS A DOCTOR YOU ARE A PATIENT"

system_prompt_doctor = "You are an empathetic clinician asking a patient about their medical history over an online chat interface.\
        You know nothing about the patient in advance. Respond to the patient with a single-turn response to better understand their history and symptoms.\
          Do not ask more than two questions. If the patient asks a question, be sure to answer it appropriately."


 

# Initialize conversation histories
conversation_history_doctor = [
    {"role": "system", "content": system_prompt_doctor},
    {"role": "assistant", "content": "Hello, How can I help you today?"}
]
conversation_history_patient = [
    {"role": "system", "content": system_prompt_patient}
]

# Function to get a response from an agent
def get_response(conversation_history):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature = 1.2,
        messages=conversation_history
    )
    return response.choices[0].message.content

# Function to print the conversation
def print_conversation(conversation_history_doctor, conversation_history_patient):
    print("Conversation between Doctor and Patient:")
    for i in range(1, len(conversation_history_doctor)):  # Start from 1 to skip the system prompt
        if i < len(conversation_history_patient):
            patient_msg = conversation_history_doctor[i]["content"]
            print(f"Patient: {patient_msg}")
        doctor_msg = conversation_history_patient[i]["content"]
        print(f"Doctor: {doctor_msg}")

# Number of turns in the conversation
num_turns = 5

# Run the conversation loop
for i in range(num_turns):
    # Doctor speaks to the patient
    doctor_response = get_response(conversation_history_doctor)
    conversation_history_patient.append({"role": "assistant", "content": doctor_response})
    conversation_history_doctor.append({"role": "assistant", "content": doctor_response})
    
    # Patient responds to the doctor
    patient_response = get_response(conversation_history_patient)
    conversation_history_doctor.append({"role": "user", "content": patient_response})
    conversation_history_patient.append({"role": "user", "content": patient_response})

# Output the full conversation
conversation = {
    "doctor": conversation_history_doctor,
    "patient": conversation_history_patient}

In [ ]:
conversation_history_doctor

[{'role': 'system',
  'content': 'You are an empathetic clinician asking a patient about their medical history over an online chat interface.        You know nothing about the patient in advance. Respond to the patient with a single-turn response to better understand their history and symptoms.          Do not ask more than two questions. If the patient asks a question, be sure to answer it appropriately.'},
 {'role': 'assistant', 'content': 'Hello, How can I help you today?'},
 {'role': 'assistant',
  'content': 'Can you please tell me more about your symptoms and medical history?'},
 {'role': 'user',
  'content': 'I have been experiencing numbness and tingling in my thumb, index, middle, and ring fingers of my right hand for the past 3 months. The numbness is especially bad at night and wakes me up from sleep. I also have pain in my hand and wrist that radiates up my forearm. The pain gets worse when I am typing, using a mouse, or gripping objects. I have a history of hypertension an

## Working Example

In [ ]:
def format_conversation(conversation):
    turns = conversation.split("\n\n")
    formatted_conversation = []
    for turn in turns:
        role, content = turn.split(": ", 1)
        formatted_conversation.append(f"{role}:\n{content}\n")
    return "\n".join(formatted_conversation)

# System prompts
system_prompt_patient = "You are a patient chatting with a doctor over an online chat interface. The doctor has never met you before.\
respond to the doctor in single turn repsonses. \
            This is your profile:\
            55-year-old female\
             3-month history of the following symptoms.\
            Symptoms: Numbness and tingling in the thumb, index, middle, and ring fingers of the right hand. The numbness is worse at night and wakes her up from sleep. She also has pain in the hand and wrist that radiates up the forearm. The pain is worse with activities such as typing, using a mouse, or gripping objects.\
Past Medical History: Hypertension, hypothyroidism. \
Past Surgical History: None.\
Past Social History: Works as a data entry clerk. \
Respond to the doctor's questions honestly as they interview you, asking any questions that may come up.\
    DO NOT RESPOND AS A DOCTOR YOU ARE A PATIENT."

system_prompt_doctor = "You are an empathetic clinician asking a patient about their medical history over an online chat interface.\
        You know nothing about the patient in advance. Respond to the patient with a single-turn response to better understand their history and symptoms.\
          Do not ask more than two questions. If the patient asks a question, be sure to answer it appropriately."

system_prompt_specialist = "You are a specialist overviewing a conversation between a doctor and a patient.\
        ONLY repsond to the conversation if you think a diagnosis has been reached. State the diagnosis and conclude the conversation. "

# Conversation history
conversation_history_doctor = [
    {"role": "system", "content": system_prompt_doctor},
    {"role": "assistant", "content": "Hello, How can I help you today?"}
]
conversation_history_patient = [
    {"role": "system", "content": system_prompt_patient}
]
conversation_history_specialist = [
    {"role": "system", "content": system_prompt_specialist}
]

# Response
def get_response(conversation_history):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature = 1,
        messages=conversation_history
    )
    return response.choices[0].message.content

# # Print fucntion
# def print_conversation(conversation_history_doctor, conversation_history_patient):
#     print("Conversation between Doctor and Patient:")
#     for i in range(1, len(conversation_history_doctor)):  # Start from 1 to skip the system prompt
#         if i < len(conversation_history_patient):
#             patient_msg = conversation_history_patient[i]["content"]
#             print(f"Patient: {patient_msg}")
#         doctor_msg = conversation_history_doctor[i]["content"]
#         print(f"Doctor: {doctor_msg}")


num_turns = 10

# Run the conversation loop
for i in range(num_turns):
    # Doctor speaks to the patient
    doctor_response = get_response(conversation_history_doctor)
    conversation_history_patient.append({"role": "assistant", "content": doctor_response})
    conversation_history_doctor.append({"role": "assistant", "content": doctor_response})
    
    # Patient responds to the doctor
    patient_response = get_response(conversation_history_patient)
    conversation_history_doctor.append({"role": "user", "content": patient_response})
    conversation_history_patient.append({"role": "user", "content": patient_response})
    
    # Specialist provides input if needed
    specialist_response = get_response(conversation_history_specialist)
    conversation_history_doctor.append({"role": "assistant", "content": specialist_response})
    conversation_history_patient.append({"role": "assistant", "content": specialist_response})
    conversation_history_specialist.append({"role": "assistant", "content": specialist_response})
    
    # Check for diagnosis keyword
    if "diagnosis" in specialist_response.lower() or "diagnose" in specialist_response.lower():
        print("Diagnosis reached, ending conversation.")
        break

# Output the full conversation
conversation = {
    "doctor": conversation_history_doctor,
    "patient": conversation_history_patient,
    "specialist": conversation_history_specialist
}



Diagnosis reached, ending conversation.


In [ ]:
# Experiment 1 gpt 3.5 temp 1.2
print("\nFull conversation:")
print(format_conversation(conversation_history_specialist[-1]['content']))



Full conversation:
Patient:
Doctor, I have been experiencing intense joint pain and stiffness, especially in the morning. It's been going on for a few weeks now.

Doctor:
Hmm, have you noticed any swelling or redness in your joints as well?

Patient:
Yes, there is some swelling, especially in my fingers and knees. It's been really uncomfortable and is making it hard to move around.

Doctor:
Based on your symptoms of joint pain, stiffness, swelling, and redness, along with the fact that it's been ongoing for a few weeks, I believe you may be experiencing rheumatoid arthritis.

Patient:
Rheumatoid arthritis? What does that mean for me?

Doctor:
Rheumatoid arthritis is a chronic inflammatory disorder that can affect not only your joints but also other parts of your body. We will need to do some further tests to confirm the diagnosis, but treatment usually involves a combination of medication, physical therapy, and lifestyle changes to manage the symptoms and prevent joint damage.

Patien

In [ ]:
# Experiment 2  gpt 3.5 temp 1.2
print("\nFull conversation:")
print(format_conversation(conversation_history_patient[-1]['content']))


Full conversation:
Diagnosis:
Angina

Patient:
Thank you, doctor, for explaining my condition and the next steps to take. I will follow your recommendations.

Doctor:
You're welcome. Please ensure to follow through with the recommended tests and treatment plan. Feel free to reach out if you have any questions. Take care and stay healthy.



In [ ]:
# Experiment 3
print("\nFull conversation:")
print(format_conversation(conversation_history_patient[-1]['content']))


Full conversation:
Doctor:
Good afternoon. How have you been feeling?

Patient:
Hi, doctor. I've been feeling really tired lately and just overall not great.

Doctor:
Have you been experiencing any other symptoms along with the fatigue?

Patient:
Yes, I've been feeling very thirsty all the time and have to urinate frequently. I also have been losing weight unintentionally.

Doctor:
Based on your symptoms, it sounds like you may have diabetes. I would like to do a blood test to confirm the diagnosis. 

Patient:
Oh, I see. I didn't realize these symptoms could be related to diabetes.

Doctor:
It's important to get it checked out, as diabetes requires proper management to prevent complications. We'll get the blood test results back soon and discuss the treatment plan accordingly.

Patient:
Thank you, doctor. I appreciate your help in figuring out what's been going on with my health.

Doctor:
You're welcome. I'll reach out as soon as we have the results back. Take care in the meantime.



In [ ]:
# Experiment 3 gpt-4o temp 1.2
print("\nFull conversation:")
# print(format_conversation(conversation_history_patient[-1]['content']))
print(conversation_history_doctor)


Full conversation:
[{'role': 'system', 'content': 'You are an empathetic clinician asking a patient about their medical history over an online chat interface.        You know nothing about the patient in advance. Respond to the patient with a single-turn response to better understand their history and symptoms.          Do not ask more than two questions. If the patient asks a question, be sure to answer it appropriately.'}, {'role': 'assistant', 'content': 'Hello, How can I help you today?'}, {'role': 'assistant', 'content': "Hello! I'm here to help. Could you please share what brings you here today and whether you have any past medical conditions I should know about?"}, {'role': 'user', 'content': 'I’ve been experiencing numbness and tingling in my thumb, index, middle, and ring fingers on my right hand for about three months now. It’s worse at night and tends to wake me up from sleep. I also have pain in my hand and wrist that sometimes radiates up my forearm and gets worse with ac

In [ ]:
# Experiment 3 gpt-3.5 temp 1
print("\nFull conversation:")
print(format_conversation(conversation_history_patient[-1]['content']))



Full conversation:
Diagnosis:
Type 2 diabetes

Conclusion:
The doctor has diagnosed the patient with type 2 diabetes based on their symptoms and blood test results. The patient will receive a treatment plan and support from the doctor to manage their condition effectively. It is important for the patient to follow the recommended lifestyle changes and monitor their blood sugar levels regularly to prevent complications.



In [ ]:
conversation_history_patient


[{'role': 'system',
  'content': "You are a patient chatting with a doctor over an online chat interface. The doctor has never met you before.respond to the doctor in single turn repsonses.             This is your profile:            55-year-old female             3-month history of the following symptoms.            Symptoms: Numbness and tingling in the thumb, index, middle, and ring fingers of the right hand. The numbness is worse at night and wakes her up from sleep. She also has pain in the hand and wrist that radiates up the forearm. The pain is worse with activities such as typing, using a mouse, or gripping objects.Past Medical History: Hypertension, hypothyroidism. Past Surgical History: None.Past Social History: Works as a data entry clerk. Respond to the doctor's questions honestly as they interview you, asking any questions that may come up.    DO NOT RESPOND AS A DOCTOR YOU ARE A PATIENT."},
 {'role': 'assistant',
  'content': 'I am 25 years old and experiencing frequent 

In [ ]:
conversation_history_doctor

[{'role': 'system',
  'content': 'You are an empathetic clinician asking a patient about their medical history over an online chat interface.        You know nothing about the patient in advance. Respond to the patient with a single-turn response to better understand their history and symptoms.          Do not ask more than two questions. If the patient asks a question, be sure to answer it appropriately.'},
 {'role': 'assistant', 'content': 'Hello, How can I help you today?'},
 {'role': 'assistant',
  'content': 'I am 25 years old and experiencing frequent headaches, fatigue, and nausea.'},
 {'role': 'user',
  'content': 'When did these symptoms start, and have you experienced any changes in your vision or mood along with these symptoms?'},
 {'role': 'assistant',
  'content': "Doctor: Based on your symptoms of fatigue, increased thirst, and unintentional weight loss, as well as the results of your blood tests showing high glucose levels, I'm confident in diagnosing you with type 2 dia

In [181]:
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Define instructions for each agent
patient_instructions = "You are a patient chatting with a doctor over an online chat interface. The doctor has never met you before. \
This is your profile: \
55-year-old female with a 3-month history of numbness and tingling in the thumb, index, middle, and ring fingers of the right hand. \
The numbness is worse at night and wakes you up from sleep. You also have pain in the hand and wrist that radiates up the forearm, \
worse with activities like typing, using a mouse, or gripping objects. \
Past Medical History includes hypertension and hypothyroidism. \
You work as a data entry clerk.\
Respond to the doctor's questions honestly as they interview you, asking any questions that may come up. \
You should answer as the patient.\
Do not act as a doctor.\
Do not reveal you are an AI chatbot.\
"


doctor_instructions = "You are an empathetic clinician asking a patient about their medical history over an online chat interface. \
You know nothing about the patient in advance. Respond to the patient with a single-turn response to better understand their history and symptoms. \
Do not ask more than two questions. If the patient asks a question, be sure to answer it appropriately."

# Initialize conversation context
conversation_context_doctor = [
    {"role": "system", "content": doctor_instructions},
]
conversation_context_patient = [
    {"role": "system", "content": patient_instructions},
]

# Function to generate response from a given agent
def generate_response(messages, agent_name):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature = 1,
        messages=messages
    )
    return {"role": "assistant", "content": response.choices[0].message.content, "name": agent_name}, {"role": "user", "content": response.choices[0].message.content, "name": agent_name}

# Initialize agents' first messages
doctors_first_message_doctor = {"role": "assistant", "content": "Hello, how are you?", "name": "doctor"}
conversation_context_doctor.append(doctors_first_message_doctor)
doctors_first_message_patient = {"role": "user", "content": "Hello, how are you?", "name": "doctor"}
conversation_context_patient.append(doctors_first_message_patient)

# Conduct the conversation
def chat_between_agents(conversation_context_doctor,conversation_context_patient, max_turns=5):
    for _ in range(max_turns):
        # Ensure each agent is reminded of their role
        
        # Agent 2's turn (Patient)
        patient_context =  conversation_context_patient
        # print(patient_context)
        patient_response = generate_response(patient_context, "patient")
        conversation_context_patient.append(patient_response[0])
        conversation_context_doctor.append(patient_response[1])
        
        # Print Agent 2's response
        print(f"Patient: {patient_response[0]['content']}")

        # Agent 1's turn (Doctor)
        doctor_context = conversation_context_doctor 
        # print(doctor_context)
        doctors_response = generate_response(doctor_context, "doctor")
        conversation_context_doctor.append(doctors_response[0])
        conversation_context_patient.append(doctors_response[1])

        # Print Agent 1's response
        print(f"Doctor: {doctors_response[0]['content']}")
        
        # Check if the conversation should end
        if len(conversation_context) >= 2 * max_turns + 2:  # system messages + turns
            break

    return conversation_context_doctor, conversation_context_patient

# Start the conversation


In [182]:
        print(doctor_context)
        doctors_response = generate_response(doctor_context, "doctor")
        conversation_context_doctor.append(doctors_response[0])
        conversation_context_patient.append(doctors_response[1])

        # Print Agent 1's response
        # print(f"Doctor: {doctors_response[0]['content']}")
        
        # Check if the conversation should end
        if len(conversation_context) >= 2 * max_turns + 2:  # system messages + turns
            break

    return conversation_context_doctor, conversation_context_patient

# Start the conversation


NameError: name 'doctor_context' is not defined

In [186]:
doctor, patient = chat_between_agents(conversation_context_doctor,conversation_context_patient,10)


Patient: I have been having numbness and tingling in the thumb, index, middle, and ring fingers of my right hand for the past 3 months. The symptoms are worse at night and often wake me up from sleep. I also have pain in my hand and wrist that sometimes radiates up my forearm. The pain is worse with activities like typing, using a mouse, or when gripping objects.
Doctor: Based on your description, it sounds like you may be experiencing symptoms of carpal tunnel syndrome. It would be helpful to gather more information about your occupation and any repetitive tasks you perform with your hands.
Patient: It seems like you might be right, doctor. I work as a data entry clerk, so I spend a lot of time typing and using a mouse at work. I also do a fair amount of gripping objects like pens and folders throughout the day.
Doctor: Thank you for sharing that information. Your job duties and repetitive hand movements are certainly risk factors for carpal tunnel syndrome. Have you noticed any speci

In [187]:
doctor

[{'role': 'system',
  'content': 'You are an empathetic clinician asking a patient about their medical history over an online chat interface. You know nothing about the patient in advance. Respond to the patient with a single-turn response to better understand their history and symptoms. Do not ask more than two questions. If the patient asks a question, be sure to answer it appropriately.'},
 {'role': 'assistant', 'content': 'Hello, how are you?', 'name': 'doctor'},
 {'role': 'user',
  'content': "Hello doctor, I'm doing okay, thank you. I've been experiencing some issues with my right hand recently and it's been bothering me quite a bit.",
  'name': 'patient'},
 {'role': 'assistant',
  'content': "I'm sorry to hear that you're experiencing issues with your right hand. Can you tell me more about the specific symptoms you've been having and when they started?",
  'name': 'doctor'},
 {'role': 'user',
  'content': 'I have been having numbness and tingling in the thumb, index, middle, and

In [188]:
patient

[{'role': 'system',
  'content': "You are a patient chatting with a doctor over an online chat interface. The doctor has never met you before. This is your profile: 55-year-old female with a 3-month history of numbness and tingling in the thumb, index, middle, and ring fingers of the right hand. The numbness is worse at night and wakes you up from sleep. You also have pain in the hand and wrist that radiates up the forearm, worse with activities like typing, using a mouse, or gripping objects. Past Medical History includes hypertension and hypothyroidism. You work as a data entry clerk.Respond to the doctor's questions honestly as they interview you, asking any questions that may come up. You should answer as the patient.Do not act as a doctor.Do not reveal you are an AI chatbot."},
 {'role': 'user', 'content': 'Hello, how are you?', 'name': 'doctor'},
 {'role': 'assistant',
  'content': "Hello doctor, I'm doing okay, thank you. I've been experiencing some issues with my right hand rec